In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the Lice`nse is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Iowa liquor data preparation

This notebook simplifies the Iowa Liquor Sales dataset to make it easier to demo.

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

### Retrieve dataset as dataframe

In [ ]:
dataset_uri = "bigquery-public-data.iowa_liquor_sales_forecasting.2020_sales_train"

In [ ]:
sql = f"""
    SELECT *
    FROM `{dataset_uri}`
"""

df = client.query(sql).to_dataframe()

### Clean data

In [ ]:
df = df.sort_values("date")
df = df.dropna()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df["county_and_city"] = df.county + " - " + df.city

### Only keep rows from counties and cities with the most rows

This is to reduce the dataset for demo purposes

In [ ]:
group = "county_and_city"

# Get zip_code_store_name with most rows
group_n_largest = df.groupby(group).sale_dollars.count().nlargest(12)

group_n_largest

In [ ]:
group_n_largest_names = group_n_largest.reset_index()[group].tolist()

group_n_largest_names

In [ ]:
df_filtered = df[df[group].isin(group_n_largest_names)]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot sales
fig = plt.gcf()
fig.set_size_inches(24, 12)

In [ ]:
sns.relplot(
    data=df_filtered,
    x="date",
    y="sale_dollars",
    # hue="project_name_and_service",
    row=group,
    height=5,
    aspect=4,
    kind="line",
    # facet_kws={'sharey': False, 'sharex': True},
    errorbar=None,
)

In [ ]:
df_filtered.to_csv("iowa_liquor_sales.csv", index=False)